####  Notes
* Use the citation network to get an idea of what papers you might want to add, especially since that'll give you a good idea of what's already linking the fields
* What am I going to DO with this dataset? Specifically problem attributes and strategies.
    * Mathematica visuals will be great
    * What other kinds of statistical analysis would be useful? 
* It's not useful unless I can get other people to use it, so how am I going to do that?
    * Pester people once I have preliminary results about how I can make this useful to them?
    * Good Github repository and readme
    * Edit Wikipedia articles?
* Citation tools: use a parser (anystyle or freecite) and then query web of science, scopus, or scifinder?
* Project defense: approved with a couple hrs of changes, vs like a week or two?

#### Code-Related Next Steps
* Put it in a github repository
* Look at connectivity on Wikipedia between network topics and graph topics

#### Notes on Crossref item properties:
* 'container-title' is the journal title, which I don't currently care about

In [3]:
# Not currently being used 
import csv 
from matplotlib import pyplot as plt

# For reading in bib.txt files
import re
import glob

# For getting the spreadsheet
import urllib
import gspread
import requests

# For using the .py database classes
import importlib
import paper
import database
import subject_assignment as subjects
import numpy as np

from collections import defaultdict
import time
from bs4 import BeautifulSoup, Comment, NavigableString

In [4]:
importlib.reload(database)
importlib.reload(paper)

db = database.Database()
db.initialize_parents("parents.txt")
#db.sync_parents()
db.rewrite_parents("parents.txt")
db.initialize_children()
#db.write_bads(<filename>)
#db.update_bads(<filename>)
db.rewrite_children()

Number of parents read from file: 221
All parents added from file; no lookup necessary
(1 of 221) Bib #1: All children added from file
(2 of 221) Bib #10: All children added from file
(3 of 221) Bib #100: All children added from file
(4 of 221) Bib #101: All children added from file
(5 of 221) Bib #102: All children added from file
DON'T ADD AN EMPTY PAPER, DUMMY
(6 of 221) Bib #103: All children added from file
(7 of 221) Bib #104: All children added from file
(8 of 221) Bib #105: All children added from file
(9 of 221) Bib #106: All children added from file
(10 of 221) Bib #107: All children added from file
(11 of 221) Bib #108: All children added from file
(12 of 221) Bib #109: All children added from file
(13 of 221) Bib #11: All children added from file
(14 of 221) Bib #110: All children added from file
(15 of 221) Bib #111: All children added from file
(16 of 221) Bib #112: All children added from file
(17 of 221) Bib #113: All children added from file
(18 of 221) Bib #114: All c

In [ ]:
db.update_bads("fixed_lastcallfornewguys.txt")

In [50]:
db.print_stats()

Total length of all reference lists: 7640
Total number of papers: 5791

	 0 unchecked and unverified

	 3823 papers unchecked
	 1968 papers manually checked
	 0 papers to get rid of
	 5791 total

	 5682 papers known to be correct
	 109 not quite right, but close enough
	 0 entries just wrong
	 5791 total


In [52]:
db.to_csv("final_database.csv")

In [6]:
importlib.reload(subjects)

subjects.clear_subjects(db)
journals = subjects.initialize_journals(db, verbose=True)
print()
topics = subjects.get_wordlists()

journals, journals_by_topic = subjects.tag_journals(db, journals, topics, 
                                                    verbose=True)
print()
papers_by_topic = subjects.tag_papers(db, journals, topics, verbose=True)
with open('journal_titles.txt','w',errors='backslashreplace') as f:
    for journal in journals:
        is_1 = subjects.check_words(journal, topics['Biology'])
        is_2 = subjects.check_words(journal, topics['Mathematics'])
        is_3 = subjects.check_words(journal, topics['Computer Science'])
        if is_1 == False and is_2 == False and is_3 == False:
            f.write(str(journal)+'\n')
            

with open('all_journal_titles.txt','w',errors='backslashreplace') as f:
    for journal in journals:
        f.write(str(journal)+'\n')

Number of papers with a container-title: 5635
Number of papers without a container-title: 156
Number of unique container titles: 2284

Total number of journals: 2284
Number of journals assigned a topic: 1502
Number of journals with no topic assigned: 782
	Number of 'Computer Science': 984
	Number of 'Biology': 296
	Number of 'Mathematics': 419

Total number of papers: 5791
Number of papers assigned a topic: 4089
Number of papers with no topic assigned: 1702
	Number of 'Computer Science': 2664
	Number of 'Biology': 1056
	Number of 'Mathematics': 901


In [25]:
num_with_word = dict()
for topic in topics:
    topic_word = dict()
    for word in topics[topic]:
        is_word = set()
        for p_hash, p in db.all_papers.items():
            p_hasword = False
            if p.container_title:
                for thing in p.container_title:
                    if subjects.check_words(thing, [word]):
                        p_hasword = True
            if p_hasword:
                is_word.add(p_hash)
        topic_word[word] = len(is_word.intersection(papers_by_topic[topic]))
    num_with_word[topic] = topic_word

In [27]:
for topic in topics:
    print(topic + ":")
    for word in topics[topic]:
        print("\t", num_with_word[topic][word], word)

Computer Science:
	 16 Machine Vision
	 72 Data Mining
	 44 Graphics
	 30 World Wide Web
	 19 Internet
	 400 Computer Science
	 15 Robotic
	 8 Electrical Engineering
	 91 SIAM
	 22 Image Analysis
	 17 Computational Intelligence
	 6 Computational linguistics
	 3 ITiCSE
	 59 Image Processing
	 12 Language Processing
	 21 learning
	 340 ACM
	 837 Computer
	 74 Artificial Intelligence
	 444 Pattern Recognition
	 52 Machine Learning
	 22 Computer Graphics
	 7 Scientific Computing
	 38 Computational Linguistics
	 329 Computer Vision
	 1 Artificial intelligence
	 4 Malware
	 82 Learning
	 82 Signal Processing
	 51 Software
	 254 Data
	 19 Machine learning
	 1 CIVR
	 14 Intelligent Systems
	 20 Document Analysis
	 59 Neural Networks
	 57 Algorithm
	 1 ITICSE
	 932 IEEE
Biology:
	 9 Drug
	 6 Microbiology
	 8 PLANT
	 11 biomedica
	 4 DNA
	 4 Biosystems
	 235 Biology
	 15 Brain
	 11 biology
	 17 Genomic
	 13 Cancer
	 25 Biomedic
	 43 Protein
	 15 Psychology
	 9 Biochem
	 1 Neurobiological
	 1 Pat

In [28]:
importlib.reload(subjects)

subjects.clear_subjects(db)
journals = subjects.initialize_journals(db, verbose=True)
print()
topics = subjects.get_wordlists()
for topic in topics:
    print(len(topics[topic]))

journals, journals_by_topic = subjects.tag_journals(db, journals, topics, 
                                                    verbose=True)

is_big3 = set()
big3 = {'Algorithm','SIAM'}
for p_hash, p in db.all_papers.items():
    p_inbig3 = False
    if p.container_title:
        for thing in p.container_title:
            if subjects.check_words(thing, big3):
                p_inbig3 = True
    if p_inbig3:
        is_big3.add(p_hash)
        
print()
papers_by_topic = subjects.tag_papers(db, journals, topics, verbose=True)
print()
print(len(is_big3))
for topic, topicpapers in papers_by_topic.items():
    print(topic, len(is_big3.intersection(topicpapers)))

Number of papers with a container-title: 5635
Number of papers without a container-title: 156
Number of unique container titles: 2284

39
43
27
Total number of journals: 2284
Number of journals assigned a topic: 1497
Number of journals with no topic assigned: 787
	Number of 'Computer Science': 984
	Number of 'Biology': 296
	Number of 'Mathematics': 393

Total number of papers: 5791
Number of papers assigned a topic: 4085
Number of papers with no topic assigned: 1706
	Number of 'Computer Science': 2664
	Number of 'Biology': 1056
	Number of 'Mathematics': 823

143
Computer Science 143
Biology 5
Mathematics 143


In [14]:
importlib.reload(subjects)

subjects.check_intersections(db, topics, papers_by_topic)

Number of papers with 0 topics: 1706
Number of papers with 1 topics: 3633
Number of papers with 2 topics: 446
Number of papers with 3 topics: 6
frozenset({'Mathematics', 'Computer Science'}) 319
frozenset({'Biology', 'Computer Science'}) 107
frozenset({'Mathematics', 'Biology'}) 20


In [44]:
importlib.reload(subjects)
hashes = subjects.get_to_read_hashes()

outfile = "./ThesisClass/thesis_bibliography.txt"

for p_hash in hashes:
    try:
        p = db.all_papers[p_hash]
        bib = requests.request('GET', 'http://dx.doi.org/' + p.DOI, 
        headers={'Accept':'application/x-bibtex'}, timeout=100)
        if bib.text.find("<") < 0:
            pass
        else:
            print("FAILURE:", p_hash)
    except KeyError:
        print("FAILED:", p_hash)

FAILED: ('Collective dynamics of Â‘small-worldÂ’ networks', 1998.0)
FAILURE: ('Efficient Graph Matching Algorithms', 1995.0)
FAILED: ('NETAL: a new graph-based method for global alignment of proteinÂ–protein interaction networks', 2013.0)
FAILURE: ('Predicting Graph Categories from Structural Properties', 2018.0)
FAILED: ('Early Estimation Model for 3DÂ–Discrete Indian Sign Language Recognition Using Graph Matching', 2018.0)
FAILURE: ('Survey on the Graph Alignment Problem and a Benchmark of Suitable Algorithms', 2013.0)
FAILURE: ('Unsupervised Domain Adaptation Using Regularized Hyper-Graph Matching', 2018.0)


In [15]:
importlib.reload(subjects)
subjects.citation_network(db, 'subject_tagged_network.gml')

7640 edges written with max_edges = 1000000.0


In [154]:
# SUBJECT IS A USELESS PIECE OF CROSSREF INFO
i = 0
with_subject = 0
without_subject = 0
subjects = set()
for p_hash, p in db.all_papers.items():
    i += 1
    if i > 10000:
        break
    """
    try:
        print(p.item['subject'])
    except KeyError:
        pass
    """
    if p.subject:
        print("\n",p.title, p.subject)
        subjects.update(set(p.subject))
        with_subject += 1
    else:
        without_subject += 1

print(with_subject)
print(without_subject)
print(len(subjects))


 Comparing performance of graph matching algorithms on huge graphs ['Signal Processing', 'Software', 'Artificial Intelligence', 'Computer Vision and Pattern Recognition']

 Indian sign language recognition using graph matching on 3D motion captured signs ['Media Technology', 'Computer Networks and Communications', 'Hardware and Architecture', 'Software']

 Accurate genotyping across variant classes and lengths using variant graphs ['Genetics']

 SSGCI: Subgraph Spotting in Graph Representations of Comic book Images ['Signal Processing', 'Software', 'Artificial Intelligence', 'Computer Vision and Pattern Recognition']

 Context-Dependent Random Walk Graph Kernels and Tree Pattern Graph Matching Kernels with Applications to Action Recognition ['Software', 'Computer Graphics and Computer-Aided Design']

 On random graphs I ['Mathematics']

 Algorithms for Graph Similarity and Subgraph Matching ['Geometry and Topology']

 An Integer Projected Fixed Point Method for Graph Matching and MAP 

In [1]:
import numpy as np

family = np.array([0.03,0.06,0.03,0.1,0.1,0.14,0.14,0.4])
comb = np.array([0.16,0.08,0.22,0.05,0.08,0.22,0.05,0.14])
trail = np.array([0.13,0.05,0.16,0.03,0.22,0.16,0.03,0.22])
helicopter = np.array([0.25,0.08,0.25,0.08,0.12,0.09,0.05,0.08])

u = np.array([100.0,70,55,50,20,15,10,0])

for action in [family, comb, trail, helicopter]:
    print(np.dot(u,action))

19.349999999999998
41.6
33.9
52.6


In [16]:
A = np.arange(12).reshape(4,3)
shifted = A - np.max(A,axis=1).reshape(4,1)
indices = np.array([0,1,2,1])
print(A)
print(np.choose(indices, A.T))
print()
print(shifted)
print()

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
[ 0  4  8 10]

[[-2 -1  0]
 [-2 -1  0]
 [-2 -1  0]
 [-2 -1  0]]



In [61]:
import itertools
import numpy as np

payoffs = {'CC':(3,3),'CD':(1,5),'DC':(5,1),'DD':(2,2)}
beta = 0.9 # discount factor
desired_reward = np.array([24.0,31.0])

for n in [2,3]:
    
    scale = 1.0/(1.0-beta**n)

    # Explore the 2^n possible strategies for a sequence of length n
    for strategy in itertools.product(('CC','CD','DC','DD'), repeat=n):

        # Construct the discounted payoffs for the infinite discounted strategy
        payoff = np.zeros((2,n))
        for i in range(n):
            payoff[:,i] = payoffs[strategy[i]]
        discount = np.array([beta**i for i in range(n)])
        
        # This might make the strategy backwards with respect to player 1 vs.
        # player 2, but it's simpler and an easy fix.
        reward = np.sort(scale*np.dot(payoff, discount))

        # Construct the enforceable and ">[24,31]" constraint
        maximin_payoff = 2.0*scale*np.sum(discount)*np.ones(2)
        is_enforceable = np.prod((reward >= maximin_payoff))
        satisfies_constraint = np.prod((reward >= desired_reward))
        
        # Report success if applicable
        if is_enforceable and satisfies_constraint:
            print("Strategy", strategy, ":", "Payoffs", np.round(reward,2))

Strategy ('CD', 'DC') : Payoffs [28.95 31.05]
Strategy ('DC', 'CD') : Payoffs [28.95 31.05]
Strategy ('CC', 'CC', 'CD') : Payoffs [24.02 35.98]
Strategy ('CC', 'CC', 'DC') : Payoffs [24.02 35.98]
Strategy ('CD', 'CC', 'DC') : Payoffs [28.6 31.4]
Strategy ('CD', 'DC', 'DC') : Payoffs [24.76 35.24]
Strategy ('DC', 'CC', 'CD') : Payoffs [28.6 31.4]
Strategy ('DC', 'CD', 'CD') : Payoffs [24.76 35.24]
